# Load historical sessions

In [1]:
import os
from dotenv import load_dotenv
import fastf1
import pandas as pd
from datetime import datetime
import sys
sys.path.append('..')
import utils
load_dotenv("../../.envrc")


True

## Load from API

In [2]:
def get_year_schedule(year):
    schedule = fastf1.get_event_schedule(year)
    return schedule

In [3]:
schedules = [
    (year, get_year_schedule(year))
    for year in range(1950, 2023)
]

api         WARNING 	

NO CACHE! Api caching has not been enabled! 
	It is highly recommended to enable this feature for much faster data loading!
	Use `fastf1.Cache.enable_cache('path/to/cache/')`



In [4]:
columns = list(schedules[-1][1].columns)
import collections
collections.Counter([list(x[1].columns)==columns for x in schedules])

Counter({True: 73})

In [5]:
df_schedules = pd.DataFrame(pd.concat([
    val.assign(Year=key) for key, val in schedules
]))

In [6]:
df_schedules

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport,Year
0,1,UK,Silverstone,,1950-05-13 00:00:00,British Grand Prix,conventional,Practice 1,1950-05-11 00:00:00,Practice 2,1950-05-11 00:00:00,Practice 3,1950-05-12 00:00:00,Qualifying,1950-05-12 00:00:00,Race,1950-05-13 00:00:00,False,1950
1,2,Monaco,Monte-Carlo,,1950-05-21 00:00:00,Monaco Grand Prix,conventional,Practice 1,1950-05-19 00:00:00,Practice 2,1950-05-19 00:00:00,Practice 3,1950-05-20 00:00:00,Qualifying,1950-05-20 00:00:00,Race,1950-05-21 00:00:00,False,1950
2,3,USA,Indianapolis,,1950-05-30 00:00:00,Indianapolis 500,conventional,Practice 1,1950-05-28 00:00:00,Practice 2,1950-05-28 00:00:00,Practice 3,1950-05-29 00:00:00,Qualifying,1950-05-29 00:00:00,Race,1950-05-30 00:00:00,False,1950
3,4,Switzerland,Bern,,1950-06-04 00:00:00,Swiss Grand Prix,conventional,Practice 1,1950-06-02 00:00:00,Practice 2,1950-06-02 00:00:00,Practice 3,1950-06-03 00:00:00,Qualifying,1950-06-03 00:00:00,Race,1950-06-04 00:00:00,False,1950
4,5,Belgium,Spa,,1950-06-18 00:00:00,Belgian Grand Prix,conventional,Practice 1,1950-06-16 00:00:00,Practice 2,1950-06-16 00:00:00,Practice 3,1950-06-17 00:00:00,Qualifying,1950-06-17 00:00:00,Race,1950-06-18 00:00:00,False,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,18,Japan,Suzuka,FORMULA 1 HONDA JAPANESE GRAND PRIX 2022,2022-10-09 16:00:00,Japanese Grand Prix,conventional,Practice 1,2022-10-07 12:00:00,Practice 2,2022-10-07 15:00:00,Practice 3,2022-10-08 12:00:00,Qualifying,2022-10-08 15:00:00,Race,2022-10-09 14:00:00,True,2022
20,19,United States,Austin,FORMULA 1 ARAMCO UNITED STATES GRAND PRIX 2022,2022-10-23 16:00:00,United States Grand Prix,conventional,Practice 1,2022-10-21 14:00:00,Practice 2,2022-10-21 17:00:00,Practice 3,2022-10-22 14:00:00,Qualifying,2022-10-22 17:00:00,Race,2022-10-23 14:00:00,True,2022
21,20,Mexico,Mexico City,FORMULA 1 GRAN PREMIO DE LA CIUDAD DE MÉXICO 2022,2022-10-30 16:00:00,Mexico City Grand Prix,conventional,Practice 1,2022-10-28 13:00:00,Practice 2,2022-10-28 16:00:00,Practice 3,2022-10-29 12:00:00,Qualifying,2022-10-29 15:00:00,Race,2022-10-30 14:00:00,True,2022
22,21,Brazil,São Paulo,FORMULA 1 HEINEKEN GRANDE PRÊMIO DE SÃO PAULO ...,2022-11-13 17:00:00,São Paulo Grand Prix,sprint,Practice 1,2022-11-11 12:30:00,Qualifying,2022-11-11 16:00:00,Practice 2,2022-11-12 12:30:00,Sprint,2022-11-12 16:30:00,Race,2022-11-13 15:00:00,True,2022


## Exporting schedules to BigQuery

In [7]:
bigquery_params = {
    "project": os.environ["BIGQUERY_PROJECT"],
    "dataset": os.environ["BIGQUERY_DATASET"],
    "credentials_path": os.environ["GOOGLE_APPLICATION_CREDENTIALS"],
    "table": "src_schedule",
    "schema": [
        ("Year", "INTEGER", "NULLABLE"),
        ("RoundNumber", "INTEGER", "NULLABLE"),
        ("Country", "STRING", "NULLABLE"),
        ("Location", "STRING", "NULLABLE"),
        ("OfficialEventName", "STRING", "NULLABLE"),
        ("EventDate", "TIMESTAMP", "NULLABLE"),
        ("EventName", "STRING", "NULLABLE"),
        ("EventFormat", "STRING", "NULLABLE"),
        ("Session1", "STRING", "NULLABLE"),
        ("Session1Date", "TIMESTAMP", "NULLABLE"),
        ("Session2", "STRING", "NULLABLE"),
        ("Session2Date", "TIMESTAMP", "NULLABLE"),
        ("Session3", "STRING", "NULLABLE"),
        ("Session3Date", "TIMESTAMP", "NULLABLE"),
        ("Session4", "STRING", "NULLABLE"),
        ("Session4Date", "TIMESTAMP", "NULLABLE"),
        ("Session5", "STRING", "NULLABLE"),
        ("Session5Date", "TIMESTAMP", "NULLABLE"),
        ("F1ApiSupport", "BOOLEAN", "NULLABLE"),
        ("uploaded_at", "TIMESTAMP", "NULLABLE"),
    ],
}

In [8]:
bq = utils.BigQuery(
    project=bigquery_params["project"],
    dataset=bigquery_params["dataset"],
    credentials_path=bigquery_params["credentials_path"],
)
bq.write_data(
    df_schedules.assign(uploaded_at=datetime.utcnow()),
    table_name=bigquery_params["table"],
    schema=bigquery_params["schema"],
)

2023-01-28 16:35:09.926 | INFO     | utils:write_data:54 - Writing 1084 rows to table tiagobbatalhao-personal:formula_one.src_schedule


LoadJob<project=tiagobbatalhao-personal, location=southamerica-east1, id=387dbc0a-f5a0-41b2-9151-1274f762abaa>